In [1]:
# New infrastructure for HMM code

In [1]:
import os
import numpy as np

from hidden import dynamics, infer
# This is only if you want to use non-default optimization routines (global)
from hidden.optimize.base import OptClass

# Initializedynamics
hmm = dynamics.HMM(2, 2)
hmm.init_uniform_cycle(0.2, 0.1)
hmm.A, hmm.B

(array([[0.8, 0.2],
        [0.2, 0.8]]),
 array([[0.9, 0.1],
        [0.1, 0.9]]))

In [2]:
# Run dynamics for 250 teps
hmm.run_dynamics(250)

In [3]:
# Get observations
obs_ts = hmm.get_obs_ts()

In [4]:
# Initialize infer object
analyzer = infer.MarkovInfer(2, 2)

In [5]:
# Now we can use initial 'guesses' for A and B to pass into the optimizer
A_init = np.array([
    [0.75, 0.3],
    [0.25, 0.7]
])

B_init = np.array([
    [0.95, 0.10],
    [0.05, 0.90]
])

In [6]:
# Now to run the optimization, we can call like this (these are all using
# default arguments, so you dont actually need to include the symmetric or
# opt_type arguments here)
result_local = analyzer.optimize(obs_ts, A_init, B_init, symmetric=False, opt_type=OptClass.Local)

In [7]:
# For a global optimization (using SHGO algorithm)
result_global = analyzer.optimize(obs_ts, A_init, B_init, symmetric=False, opt_type=OptClass.Global)

In [8]:
# ANd then if you wanted a symmetric model
A_init_sym = np.array([
    [0.75, 0.25],
    [0.25, 0.75]
])

B_init_sym = np.array([
    [0.90, 0.10],
    [0.10, 0.90]
])

result_local_sym = analyzer.optimize(obs_ts, A_init_sym, B_init_sym, symmetric=True)

In [9]:
# Now to look at the results,
result_local

In [10]:
# To see the attributes on the result we can use the '__dict__' call:
result_local.__dict__

{'_success': True,
 '_algo_name': 'L-BFGS-B',
 '_results':   message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
   success: True
    status: 0
       fun: 148.3308219706871
         x: [ 1.072e-01  1.298e-01  1.839e-01  8.838e-02]
       nit: 11
       jac: [ 3.297e-04  2.160e-04  3.496e-04  2.899e-04]
      nfev: 75
      njev: 15
  hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>,
 '_report': None,
 'likelihood': 148.3308219706871,
 '_optimal_params': array([0.10716288, 0.12980124, 0.18385793, 0.08838036]),
 'A': array([[0.87019876, 0.10716288],
        [0.12980124, 0.89283712]]),
 'B': array([[0.91161964, 0.18385793],
        [0.08838036, 0.81614207]]),
 'metadata': {}}

In [11]:
# The things we are most likely interested in are the A and B matrices, as
# well as the optimal params (which will be in the A and B matrices)

result_local.A, result_local.B

(array([[0.87019876, 0.10716288],
        [0.12980124, 0.89283712]]),
 array([[0.91161964, 0.18385793],
        [0.08838036, 0.81614207]]))

In [12]:
# These are reasonably close to the inputs (for our number of time steps)
hmm.A, hmm.B

(array([[0.8, 0.2],
        [0.2, 0.8]]),
 array([[0.9, 0.1],
        [0.1, 0.9]]))

In [14]:
# For the global optimizer, we have some more stuff in the optimization result
result_global.__dict__

{'_success': True,
 '_algo_name': 'sobol',
 '_results':  message: Optimization terminated successfully.
  success: True
      fun: 148.33082197045823
     funl: [ 1.483e+02  1.483e+02]
        x: [ 1.072e-01  1.298e-01  1.839e-01  8.838e-02]
       xl: [[ 1.072e-01  1.298e-01  1.839e-01  8.838e-02]
            [ 1.298e-01  1.072e-01  9.116e-01  8.161e-01]]
      nit: 2
     nfev: 312
    nlfev: 184
    nljev: 33
    nlhev: 0,
 '_report': None,
 'likelihood': 148.33082197045823,
 '_optimal_params': array([0.10716253, 0.12980081, 0.18385743, 0.08838008]),
 'A': array([[0.87019919, 0.10716253],
        [0.12980081, 0.89283747]]),
 'B': array([[0.91161992, 0.18385743],
        [0.08838008, 0.81614257]]),
 'metadata': {'local_min': array([[0.10716253, 0.12980081, 0.18385743, 0.08838008],
         [0.12980081, 0.10716251, 0.9116199 , 0.81614255]])}}

In [15]:
# Notable, the metatada['local_min'] which gives the local mimumum parameter values
# but we still are probably most interested in the A and B arrays

result_global.A, result_global.B

(array([[0.87019919, 0.10716253],
        [0.12980081, 0.89283747]]),
 array([[0.91161992, 0.18385743],
        [0.08838008, 0.81614257]]))

In [16]:
# And finally, the symmetric model will look very similar but only have 2 parameters
result_local_sym.__dict__

{'_success': True,
 '_algo_name': 'L-BFGS-B',
 '_results':   message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
   success: True
    status: 0
       fun: 148.87979720995418
         x: [ 1.066e-01  1.490e-01]
       nit: 10
       jac: [-8.527e-06 -4.547e-05]
      nfev: 48
      njev: 16
  hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>,
 '_report': None,
 'likelihood': 148.87979720995418,
 '_optimal_params': array([0.10663528, 0.14901276]),
 'A': array([[0.89336472, 0.10663528],
        [0.10663528, 0.89336472]]),
 'B': array([[0.85098724, 0.14901276],
        [0.14901276, 0.85098724]]),
 'metadata': {}}

In [17]:
result_local_sym.A, result_local_sym.B

(array([[0.89336472, 0.10663528],
        [0.10663528, 0.89336472]]),
 array([[0.85098724, 0.14901276],
        [0.14901276, 0.85098724]]))